Client

In [2]:
import socket
import os

# Constants
SERVER_IP = "127.0.0.1"
SERVER_PORT = 50003
FRAGMENT_SIZE = 500
WINDOW_SIZE = 10
TIMEOUT = 4
FILE_NAME = "sound_file.mp4"

# Create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client_socket.settimeout(TIMEOUT)

def send_message(message):
    client_socket.sendto(message.encode('utf-8'), (SERVER_IP, SERVER_PORT))
    print(f"Sent: {message}")

def receive_message():
    try:
        data, _ = client_socket.recvfrom(FRAGMENT_SIZE)
        print(f"Received: {data.decode('utf-8')}")
        return data.decode('utf-8')
    except socket.timeout:
        print("Timeout")
        return "TIMEOUT"

def transfer_file():
    with open(FILE_NAME, "rb") as file:
        file_size = os.path.getsize(FILE_NAME)
        print(f"File size: {file_size} bytes")

        # Send a request to start file transfer
        send_message("REQ_TO_SEND")
        response = receive_message()

        if response == "ACK":
            print(f"Sending {FILE_NAME}...")
            send_message(f"FRAGMENT_SIZE:{FRAGMENT_SIZE}")
            send_message(f"WINDOW_SIZE:{WINDOW_SIZE}")

            packet_num = 0
            seq_num = 0
            window = []

            while True:
                if seq_num < WINDOW_SIZE:
                    data = file.read(FRAGMENT_SIZE)
                    if not data:
                        break

                    # Simulate packet loss
                    if packet_num == 12 or packet_num == 55:
                        print(f"Simulating packet loss for packet {packet_num}")
                    else:
                        client_socket.sendto(data, (SERVER_IP, SERVER_PORT))
                        window.append(data)

                    seq_num += 1
                    packet_num += 1

                if seq_num == WINDOW_SIZE or not data:
                    response = receive_message()

                    if response == "ACK":
                        print("Received acknowledgment. Continuing...")
                        seq_num = 0
                        window = []
                    elif response == "NACK":
                        print("Received negative acknowledgment. Resending packets...")
                        for i, packet in enumerate(window):
                            client_socket.sendto(packet, (SERVER_IP, SERVER_PORT))
                        seq_num = 0
                    elif response == "TIMEOUT":
                        print("Server is not responding. Aborting transfer.")
                        break

    client_socket.close()

def main():
    print("** Reliable UDP Client Program **")

    # Connect to the server
    send_message("CONN_REQ")
    response = receive_message()

    if response == "ACK":
        print("Connection established!")

        while True:
            option = input("Enter 0 to send the file or 1 to terminate connection: ")

            if option == "0":
                transfer_file()
            elif option == "1":
                send_message("DISCONNECT")
                print("Connection terminated.")
                break
            else:
                print("Invalid option. Please enter 0 or 1.")

    else:
        print("Connection failed. Try again.")
if __name__ == "__main__":
    main()


** Reliable UDP Client Program **
Sent: CONN_REQ
Received: ACK
Connection established!


Enter 0 to send the file or 1 to terminate connection:  0


File size: 57951 bytes
Sent: REQ_TO_SEND
Received: ACK
Sending sound_file.mp4...
Sent: FRAGMENT_SIZE:500
Sent: WINDOW_SIZE:10


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host